# TRAINING FACE MASK DATASET

## IMPORTING NECESSARY LIBRARIES

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os


### Mounting Drive & Importing DataSet

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
INIT_LR = 1e-4
EPOCHS = 20
BS = 32
directory="/content/gdrive/MyDrive/Colab Notebooks/Face Mask dataset"
categories=["with_mask","without_mask"]


In [4]:
print("[INFO] loading images...")

data = []
labels = []
for imagePath in categories:
    path=os.path.join(directory,imagePath)
    for img in os.listdir(path):
        img_path=os.path.join(path,img)
        image = load_img(img_path, target_size=(224, 224))
        image = img_to_array(image)
        image = preprocess_input(image)
        data.append(image)
        labels.append(imagePath)



[INFO] loading images...


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


### PERFORMING ONE HOT ENCODING

In [5]:

lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

data = np.array(data, dtype="float32")
labels = np.array(labels)




### PARTIONING THE DATA INTO TRAINING AND TESTING SPLITS

In [6]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.20, stratify=labels, random_state=42)


aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")



In [7]:
baseModel = MobileNetV2(weights="imagenet", include_top=False,
                        input_tensor=Input(shape=(224, 224, 3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)



9412608/9406464 [==============================] - 0s 0us/step


In [8]:
model = Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers:
    layer.trainable = False

print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])


[INFO] compiling model...


In [9]:
print("[INFO] training head...")
H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS)

# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)



[INFO] training head...
Epoch 1/20
96/96 [==============================] - 34s 354ms/step - loss: 0.3691 - accuracy: 0.8321 - val_loss: 0.1079 - val_accuracy: 0.9727
Epoch 2/20
96/96 [==============================] - 33s 343ms/step - loss: 0.1096 - accuracy: 0.9622 - val_loss: 0.0725 - val_accuracy: 0.9779
Epoch 3/20
96/96 [==============================] - 33s 344ms/step - loss: 0.0774 - accuracy: 0.9757 - val_loss: 0.0578 - val_accuracy: 0.9805
Epoch 4/20
96/96 [==============================] - 33s 343ms/step - loss: 0.0660 - accuracy: 0.9783 - val_loss: 0.0640 - val_accuracy: 0.9753
Epoch 5/20
96/96 [==============================] - 33s 341ms/step - loss: 0.0587 - accuracy: 0.9790 - val_loss: 0.0545 - val_accuracy: 0.9792
Epoch 6/20
96/96 [==============================] - 33s 342ms/step - loss: 0.0537 - accuracy: 0.9819 - val_loss: 0.0501 - val_accuracy: 0.9805
Epoch 7/20
96/96 [==============================] - 33s 341ms/step - loss: 0.0485 - accuracy: 0.9832 - val_loss: 0.049

### CLASSIFICATION REPORT

In [10]:
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testY.argmax(axis=1), predIdxs,target_names=lb.classes_))


              precision    recall  f1-score   support

   with_mask       0.98      0.99      0.99       384
without_mask       0.99      0.98      0.99       386

    accuracy                           0.99       770
   macro avg       0.99      0.99      0.99       770
weighted avg       0.99      0.99      0.99       770

